## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## 1. Compute the camera calibration matrix and distortion coefficients using a set of chessboard images.

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    cv2.imshow('img',img)
    cv2.waitKey(100)

cv2.destroyAllWindows()

# Get calibration image shape
cal_img = cv2.imread(images[0])
cal_img_shape = cal_img.shape[1::-1]

# Calculate camera calibration params
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, cal_img_shape, None, None)

print("mtx:\n"+str(mtx))
print("dist:\n"+str(dist))

mtx:
[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128938e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
[[-0.24688775 -0.02373132 -0.00109842  0.00035108 -0.00258571]]


## 1.1. Define `cal_undistort` function to apply a distortion correction to given images with parameters previously calculated

In [2]:
# Returns undistorted image using the camera intrinsic and extrinsic parameters previously calculated
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## 1.2. Test camera calibration and distortion correction with a chessboard image

In [15]:
# Read an chessboard image
img = mpimg.imread(images[9])

# Undistort image
undistorted = cal_undistort(img, mtx, dist)

# Plot original image and the undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.0)

plt.savefig("output_images/camera_calibration.png")

# 2. Color and Gradient Threshold

## 2.1 Define `cal_threshold` function to create a thresholded binary image by applying color transform to HLS colorspace, gradient in x with Sobel operator and threshold combination.

In [28]:
# Return the combined binary image of Sobel and S Channel thresholded
def cal_threshold(img,hls_threshold=(170, 255),sobel_threshold=(20, 100),plot=False,savefig=False):

    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobel_threshold[0]) & (scaled_sobel <= sobel_threshold[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= hls_threshold[0]) & (s_channel <= hls_threshold[1])] = 1

    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    if plot:
        # Plot each HLS channel
        f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(15, 9))

        ax1.imshow(h_channel, cmap='gray')
        ax1.set_title('H Channel', fontsize=20)
        ax2.imshow(l_channel, cmap='gray')
        ax2.set_title('L Channel', fontsize=20)
        ax3.imshow(s_channel, cmap='gray')
        ax3.set_title('S Channel', fontsize=20)

        # Plot Sobel and S Channel thresholded
        ax4.imshow(sxbinary, cmap='gray')
        ax4.set_title('Sobel x Thresholded', fontsize=20)
        ax5.imshow(s_binary, cmap='gray')
        ax5.set_title('S Channel Thresholded', fontsize=20)

        # Plot the combined Sobel and S Channel thresholded image
        # ax5.imshow(color_binary)
        # ax5.set_title('Stacked', fontsize=20)
        ax6.imshow(combined_binary, cmap='gray')
        ax6.set_title('Combined', fontsize=20)

        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
        if savefig:
            plt.savefig("output_images/color_sobel_thresholds.png")

    return combined_binary

## 2.2. Test `cal_threshold()` with test images

In [27]:
# Read an chessboard image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

img_thresholded = cal_threshold(img)
img_und_thresholded = cal_threshold(img_undistorted,plot=True)

# Plot original image, the undistorted image and the respectives thresholded images
f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(12, 11))
# f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_thresholded, cmap='gray')
ax3.set_title('Thresholded Image', fontsize=15)
ax4.imshow(img_und_thresholded, cmap='gray')
ax4.set_title('Thresholded Undistorted Image', fontsize=15)

# Check differences between the original and undistorted binary thresholds
diff_binary = np.zeros_like(img_thresholded)
diff_binary[((img_thresholded == 0) & (img_und_thresholded == 1)) | ((img_thresholded == 1) & (img_und_thresholded == 0))] = 1
# diff_binary[(img_thresholded == 0) & (img_und_thresholded == 1)] = 1
color_diff_binary = np.dstack(( img_thresholded,img_und_thresholded,np.zeros_like(img_und_thresholded))) * 255

ax5.imshow(diff_binary, cmap='gray')
ax5.set_title('Binary diff Image', fontsize=15)
ax6.imshow(color_diff_binary)
ax6.set_title('Colored diff Image', fontsize=15)

plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

plt.savefig("output_images/image_thresholds.png")

# 3. Apply a perspective transform to rectify binary image ("birds-eye view").

## 3.1. Define `cal_perspective()` to return a perspective transform of the lane region

In [ ]:
def cal_perspective(img):
